# EuroSAT Baseline Deep Models

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

%matplotlib inline

## Load dataset

In [ ]:
def load_eurosat_dataset(data_dir="data"):
    data_path = Path(data_dir)

    # Get all class directories
    class_dirs = [d for d in data_path.iterdir() if d.is_dir()]
    class_names = sorted([d.name for d in class_dirs])

    print(f"Found {len(class_names)} classes: {class_names}")

    images = []
    labels = []

    # Load images from each class
    for class_idx, class_name in enumerate(class_names):
        class_path = data_path / class_name
        image_files = list(class_path.glob("*.jpg")) + list(class_path.glob("*.png"))

        print(f"Loading {len(image_files)} images from {class_name}...")

        for img_path in image_files:
            try:
                # Load image
                img = Image.open(img_path)
                img_array = np.array(img)

                # Store image and label
                images.append(img_array)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

    # Convert to numpy arrays
    data = np.array(images)
    labels = np.array(labels)

    print(f"\nDataset loaded successfully!")
    print(f"Total images           : {len(data)}")
    print(f"Data shape             : {data.shape}")
    print(f"Labels shape           : {labels.shape}")

    return data, labels, class_names


# Load the dataset
data, labels, class_names = load_eurosat_dataset("data")

## Data preprocessing

In [ ]:
# Normalize pixel values to [0, 1]
X = data.astype("float32") / 255.0
y = labels

print(f"Data shape             : {X.shape}")
print(f"Data dtype             : {X.dtype}")
print(f"Data range             : [{X.min():.2f}, {X.max():.2f}]")
print(f"Number of classes      : {len(class_names)}")

## Data split

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.13, random_state=42, stratify=y_train
)

print(f"Training set size      : {X_train.shape[0]} samples")
print(f"Test set size          : {X_test.shape[0]} samples")
print(f"Image shape            : {X_train.shape[1:]}")
print(f"Number of classes      : {len(class_names)}")

## Model training and evaluation

In [ ]:
class SparseF1Score(keras.metrics.Metric):
    """F1 Score metric that works with sparse labels."""

    def __init__(self, num_classes, average="weighted", name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.f1_metric = keras.metrics.F1Score(average=average)

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert sparse labels to one-hot encoding
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), self.num_classes)
        self.f1_metric.update_state(y_true_one_hot, y_pred, sample_weight)

    def result(self):
        return self.f1_metric.result()

    def reset_state(self):
        self.f1_metric.reset_state()

In [ ]:
def evaluate_model(
    model, X_train, X_val, y_train, y_val, model_name, epochs=20, batch_size=32
):
    print(f"\nTraining {model_name}...")

    # Train
    start_time = time.time()
    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    train_time = time.time() - start_time

    # Predict on validation set
    start_time = time.time()
    y_pred_probs = model.predict(X_val, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    predict_time = time.time() - start_time

    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average="weighted")
    recall = recall_score(y_val, y_pred, average="weighted")
    f1 = f1_score(y_val, y_pred, average="weighted")

    # Print results
    print(f"\nTraining time        : {train_time:.2f} seconds")
    print(f"Prediction time        : {predict_time:.2f} seconds")

    print(f"\nValidation Set Performance Metrics:")
    print(f"  Accuracy             : {accuracy:.4f}")
    print(f"  Precision            : {precision:.4f}")
    print(f"  Recall               : {recall:.4f}")
    print(f"  F1-Score             : {f1:.4f}")

    # Classification report
    print(f"\nClassification Report:")
    print(classification_report(y_val, y_pred, target_names=class_names))

    return {
        "model": model,
        "model_name": model_name,
        "history": history,
        "y_pred": y_pred,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "train_time": train_time,
        "predict_time": predict_time,
    }

### 1. Dense Neural Network

In [ ]:
# Build Dense Neural Network
dense_model = keras.Sequential(
    [
        layers.Input(shape=(64, 64, 3)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(len(class_names), activation="softmax"),
    ]
)

dense_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[SparseF1Score(num_classes=len(class_names), average="weighted")],
)


print("Dense Neural Network Architecture:")
dense_model.summary()

In [ ]:
# Train Dense Neural Network
dense_results = evaluate_model(
    dense_model, X_train, X_val, y_train, y_val, "Dense Neural Network", epochs=8
)

### 2. Convolutional Neural Network (CNN)

In [ ]:
# Build Convolutional Neural Network
cnn_model = keras.Sequential(
    [
        layers.Input(shape=(64, 64, 3)),
        layers.Conv2D(16, (3, 3), activation="relu"),
        layers.Conv2D(16, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(len(class_names), activation="softmax"),
    ]
)

cnn_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[SparseF1Score(num_classes=len(class_names), average="weighted")],
)


print("Convolutional Neural Network Architecture:")
cnn_model.summary()

In [ ]:
# Train Convolutional Neural Network
cnn_results = evaluate_model(
    cnn_model,
    X_train,
    X_val,
    y_train,
    y_val,
    "Convolutional Neural Network",
    epochs=8,
)

## Model comparison

In [ ]:
# Compare all models
results_summary = pd.DataFrame(
    {
        "Model": [
            dense_results["model_name"],
            cnn_results["model_name"],
        ],
        "Val Accuracy": [
            dense_results["accuracy"],
            cnn_results["accuracy"],
        ],
        "Precision": [
            dense_results["precision"],
            cnn_results["precision"],
        ],
        "Recall": [
            dense_results["recall"],
            cnn_results["recall"],
        ],
        "F1-Score": [
            dense_results["f1"],
            cnn_results["f1"],
        ],
        "Train Time (s)": [
            dense_results["train_time"],
            cnn_results["train_time"],
        ],
        "Predict Time (s)": [
            dense_results["predict_time"],
            cnn_results["predict_time"],
        ],
    }
)

print(results_summary.to_string(index=False))

In [ ]:
# Training history
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Dense NN F1 Score
axes[0, 0].plot(dense_results["history"].history["f1_score"], label="Train F1 Score")
axes[0, 0].plot(dense_results["history"].history["val_f1_score"], label="Val F1 Score")
axes[0, 0].set_xlabel("Epoch", fontsize=12, fontweight="bold")
axes[0, 0].set_ylabel("F1 Score", fontsize=12, fontweight="bold")
axes[0, 0].set_title("Dense NN - Training F1 Score", fontsize=14, fontweight="bold")
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# CNN F1 Score
axes[0, 1].plot(cnn_results["history"].history["f1_score"], label="Train F1 Score")
axes[0, 1].plot(cnn_results["history"].history["val_f1_score"], label="Val F1 Score")
axes[0, 1].set_xlabel("Epoch", fontsize=12, fontweight="bold")
axes[0, 1].set_ylabel("F1 Score", fontsize=12, fontweight="bold")
axes[0, 1].set_title("CNN - Training F1 Score", fontsize=14, fontweight="bold")
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Dense NN loss
axes[1, 0].plot(dense_results["history"].history["loss"], label="Train Loss")
axes[1, 0].plot(dense_results["history"].history["val_loss"], label="Val Loss")
axes[1, 0].set_xlabel("Epoch", fontsize=12, fontweight="bold")
axes[1, 0].set_ylabel("Loss", fontsize=12, fontweight="bold")
axes[1, 0].set_title("Dense NN - Training Loss", fontsize=14, fontweight="bold")
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# CNN loss
axes[1, 1].plot(cnn_results["history"].history["loss"], label="Train Loss")
axes[1, 1].plot(cnn_results["history"].history["val_loss"], label="Val Loss")
axes[1, 1].set_xlabel("Epoch", fontsize=12, fontweight="bold")
axes[1, 1].set_ylabel("Loss", fontsize=12, fontweight="bold")
axes[1, 1].set_title("CNN - Training Loss", fontsize=14, fontweight="bold")
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Confusion matrices

In [ ]:
def plot_all_confusion_matrices(y_val, predictions_dict, class_names):
    """Plot confusion matrices for all models in a single figure with subplots"""
    num_models = len(predictions_dict)
    fig, axes = plt.subplots(1, num_models, figsize=(12 * num_models, 10))

    if num_models == 1:
        axes = [axes]

    for idx, (model_name, y_pred) in enumerate(predictions_dict.items()):
        cm = confusion_matrix(y_val, y_pred)

        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names,
            cbar_kws={"label": "Number of samples"},
            ax=axes[idx],
        )
        axes[idx].set_xlabel("Predicted Label", fontsize=12, fontweight="bold")
        axes[idx].set_ylabel("True Label", fontsize=12, fontweight="bold")
        axes[idx].set_title(
            f"Confusion Matrix - {model_name}", fontsize=14, fontweight="bold"
        )
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45, ha="right")
        axes[idx].set_yticklabels(axes[idx].get_yticklabels(), rotation=0)

    plt.tight_layout()
    plt.show()

In [ ]:
# Confusion matrices for all models
predictions = {
    "Dense Neural Network": dense_results["y_pred"],
    "Convolutional Neural Network": cnn_results["y_pred"],
}

plot_all_confusion_matrices(y_val, predictions, class_names)